# CRIME STORY

## Dane o śmiertelnych interwencjach policji w USA

In [211]:
import pandas as pd
df = pd.read_csv('fatal-police-shootings-data.csv')
print(df)

        id                name        date   manner_of_death         armed   
0        3          Tim Elliot  2015-01-02              shot           gun  \
1        4    Lewis Lee Lembke  2015-01-02              shot           gun   
2        5  John Paul Quintero  2015-01-03  shot and Tasered       unarmed   
3        8     Matthew Hoffman  2015-01-04              shot    toy weapon   
4        9   Michael Rodriguez  2015-01-04              shot      nail gun   
...    ...                 ...         ...               ...           ...   
5411  5921      William Slyter  2020-06-13              shot           gun   
5412  5922               TK TK  2020-06-13              shot  undetermined   
5413  5924      Nicholas Hirsh  2020-06-15              shot           gun   
5414  5926               TK TK  2020-06-16              shot           gun   
5415  5927               TK TK  2020-06-16              shot           gun   

       age gender race            city state  signs_of_mental_i

## Tabela 1. Zestawienie ofiar według rasy oraz oznak choroby psychicznej. 

In [212]:
import numpy as np
df = df.pivot_table(values="name",index=['race','signs_of_mental_illness'],aggfunc=np.count_nonzero)
df = df.rename(columns={'name': 'quantity'})
print(df)

                              quantity
race signs_of_mental_illness          
A    False                          69
     True                           24
B    False                        1114
     True                          184
H    False                         745
     True                          157
N    False                          64
     True                           14
O    False                          37
     True                           11
W    False                        1763
     True                          713


## Tabela 2. Odsetek ofiar interwencji z oznakami choroby psychicznej dla każdej z ras.

In [213]:
dfs=df['quantity'].sum()
df = df.reset_index() 


def compute_percentage(row):
    total_for_race = df[df['race'] == row['race']]['quantity'].sum()
    return (row['quantity'] / total_for_race) * 100

df['percentage'] = df.apply(compute_percentage, axis=1)

print (df)


   race  signs_of_mental_illness  quantity  percentage
0     A                    False        69   74.193548
1     A                     True        24   25.806452
2     B                    False      1114   85.824345
3     B                     True       184   14.175655
4     H                    False       745   82.594235
5     H                     True       157   17.405765
6     N                    False        64   82.051282
7     N                     True        14   17.948718
8     O                    False        37   77.083333
9     O                     True        11   22.916667
10    W                    False      1763   71.203554
11    W                     True       713   28.796446


### Wniosek: Największy odsetek ofiar interwencji z oznakami choroby psychicznej jest wśród rasy białej (W).

## Tabela 3. Liczba interwencji według dni tygodnia

In [214]:
df=pd.read_csv('fatal-police-shootings-data.csv')
df['Date']= pd.to_datetime(df['date'])
df['Date'].dt.day_name().head()
dfd=df['Date'].dt.day_name().value_counts()
dfd1=pd.DataFrame(dfd)
dfd1.index.names = ['Date']
dfs=dfd1.sort_values(by='Date', ascending=True)
categs = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday', 'Sunday']
dfs = dfs.groupby(['Date']).sum().reindex(categs) 
dfs = dfs.reset_index(drop=False)
dfs.sort_values('Date')
print (dfs)

        Date  count
0     Monday    726
1    Tuesday    816
2  Wednesday    829
3   Thursday    810
4     Friday    761
5   Saturday    721
6     Sunday    753


## Wykres do tabeli 3. 

In [215]:
import xlsxwriter
writer = pd.ExcelWriter('add_chart.xlsx', engine='xlsxwriter')
dfs.to_excel(writer, sheet_name='chart')
workbook = writer.book
worksheet = writer.sheets['chart']
chart = workbook.add_chart({'type':'column'})
chart.add_series({'values':'=chart!$C$2:$C$8'})
chart.set_x_axis({
    'name': 'Days',
    'name_font':{'size': 14, 'bold': True},
    'num_font':  {'italic': True },
})
chart.set_legend({'none': True})
worksheet.insert_chart('G2', chart)
writer.close()

# # Tabela 4. Liczba icydentów w przeliczeniu na 1000 mieszkańców każdego ze stanów.

In [216]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_U.S._state_and_territory_abbreviations', header=1)[0]
df1= pd.read_html('https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population',header=0)[0]
dfa=pd.DataFrame(df)
dfb=pd.DataFrame(df1)
dfar=dfa.drop(axis=0,index=[0,1,2,3,4,5,6,7,8,9])
dfac=dfar.drop(axis=1, labels=['Codes:.1','Unnamed: 2','Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7','Unnamed: 8', 'Unnamed: 9'])
dfar2=dfac.dropna(axis=0)
dfarr=dfar2.rename(columns={'Codes:':'State','Unnamed: 3':'Abbrev'})
dfarr2=dfarr.sort_values('Abbrev')
dfarr3=dfarr2.drop(axis=0,index=[66,76,77,78])

df1s=df1.sort_values('State')
df1s.dropna(axis=0, inplace=True)
dfac2=df1s.drop(axis=1, labels=['Rank in states & territories, 2020','Rank in states & territories, 2010'])
dfac3=dfac2.drop(axis=1, labels=['Census population, April 1, 2010 [1][2]','Percent change, 2010–2020 [note 1]','Absolute change, 2010-2020','Total seats in the U.S. House of Representatives, 2023–2033','Census population per electoral vote [note 2]','Census population per House seat','Percent of the total U.S. population, 2020 [note 3]'])
dfar3=pd.DataFrame(dfarr3)
dfac4=pd.DataFrame(dfac3)


def make_keys(n):
    return [f'key{n}' for n in range(n)]
left = dfar3.copy()
left['key'] = make_keys(56)
right = dfac4.copy()
right['key'] = make_keys(56)

dfm=pd.merge(left,right,how='inner',on='key')
dfm1=dfm.drop(columns=['key','State_y'])
dfm2=dfm1.rename(columns={'Census population, April 1, 2020 [1][2]':'Population 2020'})
dfm2

,State_x,Abbrev,Population 2020
0,Alaska,AK,5024279
1,Alabama,AL,733391
2,Arkansas,AR,49710
3,American Samoa,AS,7151502
4,Arizona,AZ,3011524
5,California,CA,39538223
6,Colorado,CO,5773714
7,Connecticut,CT,3605944
8,District of Columbia,DC,989948
9,Delaware,DE,689545


In [217]:
import pandas as pd
df = pd.read_csv('fatal-police-shootings-data.csv')
dfst=df['state'].value_counts()
dfst1=pd.DataFrame(dfst)
dfsv=dfst1.sort_values(by='state', ascending=True)
dfm2.dropna()
dfn=dfm2['Total shoot 2020']=dfsv['count']
dfn1=dfn.dropna().astype(int)
pddfn1=pd.DataFrame(dfn1)

dfn2 = pddfn1.rename(columns={'count':'Total Shootings'})


def make_keys(n):
    return [f'key{n}' for n in range(n)]
left = dfm2.copy()
left['key'] = make_keys(56)
right = dfn2.copy()
right['key'] = make_keys(51)

dfmn=pd.merge(left,right,how='inner',on='key')
dfmn
dfmn2=dfmn.drop(columns=['Total shoot 2020','key'])
dfmn2

def shoot_per_1000(row):
    return (row['Total Shootings'] * 1000/ row['Population 2020'])

dfmn2['Shootings per 1000'] = dfmn2.apply(shoot_per_1000, axis=1)

dfmn2



,State_x,Abbrev,Population 2020,Total Shootings,Shootings per 1000
0,Alaska,AK,5024279,39,0.007762
1,Alabama,AL,733391,104,0.141807
2,Arkansas,AR,49710,83,1.669684
3,American Samoa,AS,7151502,254,0.035517
4,Arizona,AZ,3011524,799,0.265314
5,California,CA,39538223,195,0.004932
6,Colorado,CO,5773714,21,0.003637
7,Connecticut,CT,3605944,13,0.003605
8,District of Columbia,DC,989948,13,0.013132
9,Delaware,DE,689545,350,0.507581
